In [2]:
import pandas as pd
import numpy as np

## Задание 0


Для всех  заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **tr_mcc_codes, tr_types, transactions и gender_train** из одноимённых таблиц из папки [data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L).

Для таблицы transactions используйте только первые n=1000000 строк. При чтении файлов обратите внимание на разделители внутри каждого из файлов - они могут различаться!

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

download = drive.CreateFile({'id' : '10J8RzMIhoYHiad49r-oWNMAk-V5lo3OE'})
download.GetContentFile('tr_mcc_codes.csv')
tr_mcc_codes = pd.read_csv('tr_mcc_codes.csv',sep=';')

download = drive.CreateFile({'id' : '1lEpoRKczv5EvZhff9O2I-0JkdHnbe_Mq'})
download.GetContentFile('transactions.csv')
transactions = pd.read_csv('transactions.csv', nrows=100000, sep =',')

download = drive.CreateFile({'id' : '1FG1fopcmvMZ7GBaBOqQipccSeFoMUvNT'})
download.GetContentFile('gender_train.csv')
gender_train = pd.read_csv('gender_train.csv')

download = drive.CreateFile({'id' : '1EP3KrATWS1I_qYdpRhYnSDl-eoBiOBQc'})
download.GetContentFile('tr_types.csv')
tr_types = pd.read_csv('tr_types.csv', sep=';')

###  Описание данных
#### Таблица ```transactions.csv```
##### Описание
Таблица содержит историю транзакций клиентов банка за один год и три месяца.

##### Формат данных

```
customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
111111,15 01:40:52,1111,1000,-5224,111111
111112,15 15:18:32,3333,2000,-100,11122233
...
```
##### Описание полей

 - ```customer_id``` — идентификатор клиента;
 - ```tr_datetime``` — день и время совершения транзакции (дни нумеруются с начала данных);
 - ```mcc_code``` — mcc-код транзакции;
 - ```tr_type``` — тип транзакции;
 - ```amount``` — сумма транзакции в условных единицах со знаком; ```+``` — начисление средств клиенту (приходная транзакция), ```-``` — списание средств (расходная транзакция);
 - ```term_id``` — идентификатор терминала;


#### Таблица ```gender_train.csv```

##### Описание
Данная таблица содержит информацию по полу для части клиентов, для которых он известен. Для остальных клиентов пол неизвестен.

##### Формат данных
```
customer_id,gender
111111,0
111112,1
...
```

##### Описание полей
 - ```customer_id``` — идентификатор клиента;
 - ```gender``` — пол клиента;

### Таблица ```tr_mcc_codes.csv```

##### Описание
Данная таблица содержит описание mcc-кодов транзакций.

##### Формат данных
```
mcc_code;mcc_description
1000;словесное описание mcc-кода 1000
2000;словесное описание mcc-кода 2000
...
```

##### Описание полей
 - ```mcc_code``` – mcc-код транзакции;
 - ```mcc_description``` — описание mcc-кода транзакции.

#### Таблица ```tr_types.csv```

##### Описание
Данная таблица содержит описание типов транзакций.

##### Формат данных
```
tr_type;tr_description
1000;словесное описание типа транзакции 1000
2000;словесное описание типа транзакции 2000
...
```

##### Описание полей
 - ```tr_type``` – тип транзакции;
 - ```tr_description``` — описание типа транзакции;


Задания 1-4 делаются без использования merge!

## Задание 1



1. Для столбца tr_type датафрейма transactions выберите произвольные 1000 строк с помощью метода sample
2. В полученной на предыдущем этапе подвыборке найдите долю транзакций (стобец tr_description в датасете tr_types), в которой содержится подстрока 'POS' или 'ATM'


In [4]:
transactions_result = transactions['tr_type'].sample(n=1000)
selected = tr_types[tr_types["tr_description"].str.contains('POS') | tr_types["tr_description"].str.contains('АТМ')]['tr_type']
count = transactions_result.isin(selected).value_counts()
result = round(count[True] / 1000, 4)
print(result)





0.667


## Задание 2


1. Для столбца tr_type датафрейма transactions посчитайте частоту встречаемости всех типов транзакций tr_type в transactions.
2. Выведите топ-10 транзакций по частоте встречаемости (вывести для них tr_description тоже).

In [20]:
transaction_counts = transactions['tr_type'].value_counts()/100000

transaction_counts.sort_values(ascending=False)
print(transaction_counts.head(10))

description_mode = tr_types[tr_types['tr_type'].isin(transaction_counts.head(10).index)]
print(description_mode)


1010    0.27149
2010    0.17305
1110    0.15167
1030    0.13499
7070    0.06277
2370    0.06090
7010    0.03039
1100    0.01762
1200    0.01280
7030    0.01035
Name: tr_type, dtype: float64
     tr_type                                     tr_description
39      7010       Взнос наличных через АТМ (в своем тер.банке)
47      7030  Перевод на карту (с карты) через АТМ (в предел...
59      7070  Перевод на карту (с карты) через Мобильный бан...
98      1010                              Покупка. POS ТУ СБ РФ
99      1030                     Оплата услуги. Банкоматы СБ РФ
100     1100                                Покупка. ТУ  Россия
101     1110                             Покупка. POS ТУ Россия
102     1200                               Покупка. Зарубеж. ТУ
106     2010              Выдача наличных в АТМ Сбербанк России
129     2370  Списание с карты на карту по операции <перевод...


## Задание 3
1. В датафрейме transactions найдите клиента с максимальной суммой приходов на карту
2. В датафрейме transactions найдите клиента с максимальной суммой расходов по карте
3. Найдите модуль разницы для этих клиентов между суммой расходов и суммой приходов

In [42]:
maxSum = transactions[transactions.amount>0].groupby(by=['customer_id']).sum().sort_values(by=['amount'], ascending=False).head(1)
print("Max Sum: ", maxSum.index.values[0])

minSum = transactions[transactions.amount < 0].groupby(by=['customer_id']).sum().sort_values(by=['amount'], ascending=True).head(1)
print("Min Sum: ", minSum.index.values[0])

difference = abs(maxSum['amount'].values[0] + minSum['amount'].values[0])
difference

Max Sum:  47853460
Min Sum:  47853460


<ipython-input-42-d00e6f639fa8>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  maxSum = transactions[transactions.amount>0].groupby(by=['customer_id']).sum().sort_values(by=['amount'], ascending=False).head(1)
<ipython-input-42-d00e6f639fa8>:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  minSum = transactions[transactions.amount < 0].groupby(by=['customer_id']).sum().sort_values(by=['amount'], ascending=True).head(1)


22549870.78000003

## Задание 4
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций из топ 10 из задания 2
1. Найдите среднее арифметическое и медиану по amount по всем типам транзакций для клиентов из задания 3

In [52]:
current_transactions = transactions[transactions['tr_type'].isin(description_mode['tr_type'])]

means = current_transactions[['tr_type', 'amount']].groupby('tr_type').mean('amount')
medians = current_transactions[['tr_type', 'amount']].groupby('tr_type').median('amount')

print('Means:', means)
print('Medians:', medians)

Means:                 amount
tr_type               
1010     -22863.403916
1030      -5480.064844
1100     -50347.348127
1110     -41019.509332
1200     -52027.017188
2010    -125578.163257
2370    -213480.996626
7010     346718.698295
7030     179112.232106
7070     157992.110871
Medians:             amount
tr_type           
1010      -7808.60
1030      -2245.92
1100     -11229.58
1110     -11802.29
1200     -10854.40
2010     -44918.32
2370     -44918.32
7010     112295.79
7030      26075.08
7070      22459.16


## Подготовка для заданий 5-8

*Из заданий 5-8 нужно выполнить минимум два любых*

Соедините transactions с всеми остальными таблицами (tr_mcc_codes, tr_types, gender_train). Причём с gender_train необходимо смёрджиться с помощью left join, а с оставшимися датафреймами - через inner.
После получения результата таблицы gender_train, tr_types, tr_mcc_codes можно удалить. В результате соединения датафреймов должно получиться 999584 строки.

In [56]:
transactions = pd.merge(transactions, gender_train, how='left')
transactions = pd.merge(transactions, tr_mcc_codes, how='inner')
transactions = pd.merge(transactions, tr_types, how='inner')
transactions.shape

(99992, 9)

## Задание 5

1. Определите модуль разницы между средними тратами женщин и мужчин (трата - отрицательное значение amount).
2. Определите модуль разницы между средними поступлениями у мужчин и женщин

Обратите внимание, что для вычисления модуля разности точных знаний о том,
какой класc относится к мужчинам, а какой - к женщинам, не требуется.

In [75]:
gender_1 = transactions[transactions.gender == 1]
gender_0 = transactions[transactions.gender == 0]
gender_1_expenses = gender_1[gender_1['amount'] < 0].mean()
gender_0_expenses = gender_0[gender_0['amount'] < 0].mean()

difference_expenses = abs(gender_1_expenses-gender_0_expenses)

print("difference_expenses ",difference_expenses.amount)

gender_1 = transactions[transactions.gender == 1]
gender_0 = transactions[transactions.gender == 0]
gender_1_receipts = gender_1[gender_1['amount'] > 0].mean()
gender_0_receipts = gender_0[gender_0['amount'] > 0].mean()

difference_receipts = abs(gender_1_receipts-gender_0_receipts)

print("difference_receipts ",difference_receipts.amount)


<ipython-input-75-a38d012c5cfa>:3: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  gender_1_expenses = gender_1[gender_1['amount'] < 0].mean()
<ipython-input-75-a38d012c5cfa>:4: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  gender_0_expenses = gender_0[gender_0['amount'] < 0].mean()


difference_expenses  20819.298191988695
difference_receipts  63813.95012566788


<ipython-input-75-a38d012c5cfa>:12: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  gender_1_receipts = gender_1[gender_1['amount'] > 0].mean()
<ipython-input-75-a38d012c5cfa>:13: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  gender_0_receipts = gender_0[gender_0['amount'] > 0].mean()


## Задание 6

1. По всем типам транзакций рассчитайте максимальную сумму прихода на карту (из строго положительных сумм по столбцу amount) отдельно для мужчин и женщин (назовите ее "max_income"). Оставьте по 10 типов транзакций для мужчин и для женщин, наименьших среди всех типов транзакций по полученным значениям "max_income".
2. Выделите среди них те типы транзакций, которые встречаются одновременно и у мужчин, и у женщин

In [90]:
gender_0_income = gender_0[gender_0['amount']>0][['tr_type', 'amount']]
gender_1_income = gender_1[gender_1['amount']>0][['tr_type', 'amount']]

gender_0_max_income = gender_0_income.groupby(by='tr_type').max('amount').sort_values(by='amount').head(10)
gender_1_max_income = gender_1_income.groupby(by='tr_type').max('amount').sort_values(by='amount').head(10)

print(gender_0_max_income)

print(gender_1_max_income)

coincidences = gender_0_max_income[gender_0_max_income.index.isin(gender_1_max_income.index)]
print(coincidences)

             amount
tr_type            
4100        5041.86
7035       49410.15
1110       51341.63
6200      204194.17
6010      313979.02
8145      320145.86
6100      369848.43
7014     1010662.10
7011     1122957.89
6110     1122957.89
             amount
tr_type            
6200       60119.35
6000       79280.83
6010      119033.54
1110      125187.35
1100      134530.35
7034      224591.58
7074      404264.84
6100      469598.53
7014      920825.47
7024     1251861.32
             amount
tr_type            
1110       51341.63
6200      204194.17
6010      313979.02
6100      369848.43
7014     1010662.10


## Задание 7

1. Найдите суммы затрат по каждой категории (mcc) для мужчин и для женщин.
2. Найдите топ 10 категорий с самыми большими относительными модулями разности в тратах для разных полов (в ответе должны присутствовать описания mcc кодов).

## Задание 8

1. Из поля tr_datetime выделите час tr_hour, в который произошла транзакция, как первые 2 цифры до ":". (\**)
2. Посчитайте количество транзакций с amount<0 в ночное время для мужчин и женщин. Ночное время - это примерно 00-06 часов.